In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def census(target,lis):
    answer=0
    for n in lis:
        if n==target:answer=answer+1
    return answer

def positions(target,lst):
    ln=len(lst)
    answer=[]
    for j in [0..ln-1]:
        entry=lst[j]
        if entry==target:answer=answer+[j+1]
    return(answer)

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
#Broadhurst code from #A005148:

#? {nt=100; a=[1]; b=[1]; d=1; e=0; g=0; print(1); for(n=2, nt, c=48*(a[n-1]+g)+128*(d-32*e); 
#e=d; d=c; i=(n-1)\2; g=12*if(n%2==0, a[n/2]^2)+24*sum(j=1, i, 
#a[j]*a[n-j]); h=12*if(n%2==0, b[n/2]^2)+24*sum(j=1, i, b[j]*b[n-j]); f=(c+5*h)/n^2-g; a=concat(a, f); 
#b=concat(b, n*f); print(f))}

#produces this on the PARI online evaluator (I have inserted the commas):
br=[
1,
47,
2488,
138799,
7976456,
467232200,
27736348480,
1662803271215,
100442427373480,
6103747246289272,
372725876150863808,
22852464771010647496,
1405886026610765892544,
86741060172969340021952,
5365190340823180439326208,
332577246704242939511725615,
20655377769544663820919905000,
1285027807539621869480480977880,
80066610886753513409821525593280,
4995543732566526565060187887772024,
312067903389730540416319245145039936,
19516459352109724206910675815791735872,
1221787478073080268912138739833447254528,
76558881238278398609546573647116818306504,
4801399849802188285872546222298724299377856,
301358552889212442951924121355286655092791360,
18928524108186605379268259069278244869735006720,
1189719542605042010945455887482239233732751142080,
74824958481405101799295401923145498080031496317440,
4708731584940969251488540213411242070133095720768000,
296483323638911778793802123013217365155428610625064960,
18677571039055424502042574350078071038555962934810664495,
1177200955467256907707767829606512556434525730284672082280,
74229820742983998523807878655148660941364964757170232076440,
4682657672641000613276353688819373189604961982881761635174080,
295516785862704112676947743865736338547152307208873658542187480,
18656838683258040776726836797753969443154060448210951169536087360,
1178287550937265649491805466460363896744099593833261406542090821440,
74441259433548426510664621182339422182178689134172479673100078686720,
4704546876230537649051669928635037299315044055233418643313504347890040,
297410696380227510473584821926459754598587577997951261584830786025989440,
18807176292551896455842616399574167855948518855982280636468413444438841280,
1189632505858785415664268185396568316810012962868095237190924015678644805120,
75269436592700558660145646818728077669744495747378078929068356710829357904960,
4763606735739477078702262301306618196904330454342036172567804617626114845601280,
301550219357655322958904198748139655940272138707157414253528789096123355242370560,
19093491105382437947961430595496009051927469794600124607374594862297809973497425920,
1209229421833128214532165231904398024088456532579184673374765702204525386892709582280,
76599462222171488217469562807555444840329820375936645628428503967599842536403748392640,
4853249476279584943018752544135518205835823652569328104071808597099976302206777672382272,
307557161322636240130429378891312616501367843861094990590342588813640658716687276702796288,
19494049310445477564550759598016449348244789590449694853790210431694353584090244058674665536,
1235828235382992966112000337356874542591602729215314383988292324949451319922660010288507026944,
78359386541981263753587663605771220395173695760309511724942887619218844604408365062990458445312,
4969331730256147172835336645727281480799885420753251882375600240456687485460922105379475213365248,
315192836223790555211163444188645825759965673786927108224693473418739558121668975477385609064230080,
19995098780164839843271975968833674908892528077762766665489021901583285029365260596621852077438476800,
1268636632413268672729488723673519658349225434504337043331170274695193536624267473803997222986457986560,
80503572319005599896519802085748511261953758070993177770252662999751870126880338970620619030455487016960,
5109226023515231795174271806642022238278119546486922652746346463140324125835625762209193286894097317188096,
324306079896787161344357124063601140357997183120543361083780244695916059297903737976316852087325736124084224,
20587951238469144728875413572545165973124146169862711202444824611729329948630917798996527215521428049827442688,
1307155886598940172607280751052393343723376964347599361863473016335357927316478471270092962849033836059872428032,
83003437890819298061159220674050513337514687509034267017635549288525596646068731914307086031798152640655708675631,
5271297646867701243231620056348828996858362139183014747261830107569871776680492045283621280463257824831450361228904,
334803625831811164273307471398151444570380979015726639675077730502474540099263374815508109958714263034979870045590040,
21267301347069209180434546119316800827596753601228462222166318226579132663526299067285136882052633679235269979706997440,
1351085301766957831759285167329838127692235945761375100182242807152550787271816136969058195839695684514251809964546971800,
85842015014782910888737358332857326909867676530325317899274801087388530640486646412863052257542933472850576480889534912320,
5454594181103314395353884318297045062026694474478123129221277721208935931363965378785241897523306877686001373372245607417152,
346632363267943630179319892957146981236491202095299908808662717047804243504180203803669349935923567992160546743779715198734848,
22030211252839896655760044436342222596667157343136792244243996326108725373610974640845764035532176284454550965374859141605876696,
1400264005045227498693320089227035884825680282492704474103652886750979292873550448366299419926933438094105780187281431855082851904,
89010607580108363146057947796447019984406819117601908324673767705214888723143847657440514237648023072092581303730690468541358446272,
5658653431087770147783431973072442739707731119479989465299403213540757386414610090412270459203277370032300491403324236240673058231808,
359768282283136451938233183643880156808956764689158714079901896700007719423392215459057530585195812985027222576505521483746145927094080,
22875473496936829687402292014161718834412927763836095316924893494178776078421448526507427925257601899589958056800447532066174066611688960,
1454634188002381701560543002374509285000151863499255133979293536524595484779236558931041033869616374153934389337102738325693541500885040640,
92506668310411752697700027500541107599054510647853447997767226456825933551220718898889759216482942932408592031625495444638507054727309701120,
5883380647402071727818124358464357357061997088366913641247755161122708769301392216445412497947130792332367814389247083022141893681918673035640,
374209367072642703061850255891790471856520862005092473938011251040764312378697398309875157737634899606364054132972840806767260748645213681887040,
23803199270417251468737963122626707467816366171461192749563400557478169477201165362512851772159448598935076348960799452562007578125691327998310080,
1514217230105491006416286735192399705713778112569360921940868647785308212475854144353099514116567802127049424293324092083010142613089652094239352320,
96332413045374746200370571514086365134691986056054938357641223739177015618938978964850794817403946266783867698429445521201166592631737075434674509760,
6128968041798698464871983060993177701445048464359248049771600533082817745158394358756109989641756654653420473533147391458535031474858708357818557350400,
389970917973786025826826148232023527188983145106526374860563378061488842725970203994300386046448714269543873579140545202535929354547230426525139471434240,
24814546341676335548842726357075976008312393230217108797033854347814848781823641548406242672165084256308341602099263161325277248861481206883551599766753280,
1579097865621621078127966349210038916312383695762720347829578839794195077696910123713050602263102661325782769915208166009373762048848807388370046579798291520,
100493898862353192639599085772479279507605308104393463045761183781449039477596456154787934584841158653853713852389459365917827655606822211804400142188630504960,
6395841086583513721518708585123859851483827511052356442692606484016955654747071476807355907729334535301712240745743084597082858081715137457733426771133436244480,
407082422122816769108928976084224767774083074925265772652509091404070682745260601092117551854989868576150378265801324011663291789547097044490460289150388722831360,
25911536639811395996751656925990326528992266850810482361355730044302538357241007798389242049209348961586470395969302758332164645822911593670231053397678686836037120,
1649413547559894591700394303117536943729317363777978884992676745553087694293098678096611231647446457954863017134436134803435906819926459313995976199288693719229255680,
105000403824773770333649105789820480034335517728840968988334310643726824123735041790377778857595055184234699203779438530862092851011368656739154398110404364696677847040,
6684622341158487811999668275612452018461145981041862416511208589860402818472403829783593014653257049064920715626593324371966426230798803277918448269747031588460412436480,
425585443813049916737778209891287760908166309712249413374261870396364033955750883326966977701620958916811619741413201896787531939944010422188516605008297556440177053390280,
27096933232548903092613115617140470313230857234611666991627504070866204307527763605986822544358886296299818124185350044201435781986714935884426869011696899042312957811757760,
1725347275200146778788065707502165945462707429554800950293313680035690477290460889039398857128345130378453865634762702538349329424603778897699271642610584658725955102258591040,
109864008941653383497254010859226498595190791378492674731068039950918263737791469856053624392850573136737164626778125500188292591996295723267255817207828065983109119673561029120,
6996107097499132971946127571962655485323611430800974100070703369809020527942862074490096967477739069312314066266444140870911945979977377211097174725520015644037734486845682825024]

def broadhurst(n):return br[n-1]

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [37]:
# verify narrow version, p = 5
import time

def p5RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=a*sigma(a,k=9)
        ans=ans+term*x^a
    return D(ans)

def recipP5(n):
    ans=L(1/p5RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p5RecipPowerConstantTerm(k):
    g=recipP5(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

one=[]
zero=[]
od=[]
no=[]
start=time.time()
for a in [1..200]: #default precision must be > max a.
    c=p5RecipPowerConstantTerm(a)
    o5=order(5,c)
    od=od+[o5]
    finish=time.time()
    print((a,o5,finish-start))
print();print("od:");print(od)
print()
ps=positions(0,od);print(ps) 
#A037453 Positive numbers whose base-5 representation contains no 3 or 4.

(1, 0, 0.018056869506835938)
(2, 0, 0.0418848991394043)
(3, 1, 0.07126498222351074)
(4, 1, 0.1065070629119873)
(5, 0, 0.14688491821289062)
(6, 0, 0.21280574798583984)
(7, 0, 0.27875304222106934)
(8, 2, 0.3421499729156494)
(9, 1, 0.4045870304107666)
(10, 0, 0.47165584564208984)
(11, 0, 0.5352919101715088)
(12, 0, 0.6016020774841309)
(13, 1, 0.6668107509613037)
(14, 3, 0.7309298515319824)
(15, 1, 0.8005850315093994)
(16, 1, 0.8652667999267578)
(17, 4, 0.9289710521697998)
(18, 2, 0.9921789169311523)
(19, 2, 1.0555410385131836)
(20, 1, 1.120910882949829)
(21, 1, 1.1911439895629883)
(22, 2, 1.2656567096710205)
(23, 2, 1.337336778640747)
(24, 2, 1.4081058502197266)
(25, 0, 1.477799892425537)
(26, 0, 1.5473520755767822)
(27, 0, 1.616724967956543)
(28, 1, 1.6873009204864502)
(29, 1, 1.762274980545044)
(30, 0, 1.836181879043579)
(31, 0, 1.909454107284546)
(32, 0, 1.979882001876831)
(33, 2, 2.055790901184082)
(34, 1, 2.127429962158203)
(35, 0, 2.199054002761841)
(36, 0, 2.270658016204834)
(37, 0

In [39]:
ps=positions(0,od);print(ps) #A037453 Positive numbers whose base-5 representation contains no 3 or 4.

[1, 2, 5, 6, 7, 10, 11, 12, 25, 26, 27, 30, 31, 32, 35, 36, 37, 50, 51, 52, 55, 56, 57, 60, 61, 62, 125, 126, 127, 130, 131, 132, 135, 136, 137, 150, 151, 152, 155, 156, 157, 160, 161, 162, 175, 176, 177, 180, 181, 182, 185, 186, 187]


In [6]:
# verify narrow version, p = 5
import time

def p5RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=rmnjntau(a)
        ans=ans+term*x^a
    return D(ans)

def recipP5(n):
    ans=L(1/p5RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p5RecipPowerConstantTerm(k):
    g=recipP5(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

A037453=[]
zeros=[]
od=[]
start=time.time()
for a in [1..20]: #default precision must be > max a.
    c=p5RecipPowerConstantTerm(a)
    o5=order(5,c)
    if o5==0:zeros=zeros+[a]
    od=od+[o5]
    c3=census(3,a.digits(5))
    c4=census(4,a.digits(5))
    if c3==0:
        if c4==0:
              A037453=A037453+[a]
    finish=time.time()
    print((a,finish-start))
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(A037453);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)
# ps is A037453		Positive numbers whose base-5 representation contains no 3 or 4.

(1, 0.008862018585205078)
(2, 0.018215179443359375)
(3, 0.028209209442138672)
(4, 0.03885316848754883)
(5, 0.04885292053222656)
(6, 0.06247591972351074)
(7, 0.07741594314575195)
(8, 0.09165406227111816)
(9, 0.10671806335449219)
(10, 0.12079405784606934)
(11, 0.1359100341796875)
(12, 0.15085601806640625)
(13, 0.1651170253753662)
(14, 0.17900896072387695)
(15, 0.1928260326385498)
(16, 0.20660996437072754)
(17, 0.22060585021972656)
(18, 0.23406219482421875)
(19, 0.24744510650634766)
(20, 0.2612450122833252)

od:
[0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 5, 2, 2, 1]

vno-vps:

(0, 0, 0, 0, 0, 0, 0, 0)


In [7]:
# verify narrow version, p = 5
import time

def p5RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=rmnjntau(a)
        ans=ans+term*x^a
    return D(ans)

def recipP5(n):
    ans=L(1/p5RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p5RecipPowerConstantTerm(k):
    g=recipP5(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

A037453=[]
zeros=[]
od=[]
start=time.time()
for a in [1..1000]: #default precision must be > max a.
    c=p5RecipPowerConstantTerm(a)
    o5=order(5,c)
    if o5==0:zeros=zeros+[a]
    od=od+[o5]
    c3=census(3,a.digits(5))
    c4=census(4,a.digits(5))
    if c3==0:
        if c4==0:
              A037453=A037453+[a]
    finish=time.time()
    print((a,finish-start))
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(A037453);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)
# ps is A037453		Positive numbers whose base-5 representation contains no 3 or 4.

(1, 0.008013010025024414)
(2, 0.018597126007080078)
(3, 0.029034852981567383)
(4, 0.039999961853027344)
(5, 0.04979705810546875)
(6, 0.0634310245513916)
(7, 0.07859110832214355)
(8, 0.09302806854248047)
(9, 0.10766911506652832)
(10, 0.12174510955810547)
(11, 0.13582515716552734)
(12, 0.15089011192321777)
(13, 0.16495704650878906)
(14, 0.17888617515563965)
(15, 0.1932380199432373)
(16, 0.20761585235595703)
(17, 0.2222881317138672)
(18, 0.2362051010131836)
(19, 0.2501978874206543)
(20, 0.2646920680999756)
(21, 0.28092408180236816)
(22, 0.30246400833129883)
(23, 0.3180699348449707)
(24, 0.33386993408203125)
(25, 0.350750207901001)
(26, 0.36804914474487305)
(27, 0.386066198348999)
(28, 0.40426206588745117)
(29, 0.4231300354003906)
(30, 0.4427480697631836)
(31, 0.4652903079986572)
(32, 0.48687005043029785)
(33, 0.5064470767974854)
(34, 0.5277302265167236)
(35, 0.5487921237945557)
(36, 0.5700521469116211)
(37, 0.5923519134521484)
(38, 0.6162810325622559)
(39, 0.6406528949737549)
(40, 0.66464

(327, 108.58688712120056)
(328, 109.54003810882568)
(329, 110.48770022392273)
(330, 111.44994115829468)
(331, 112.44064497947693)
(332, 113.4591772556305)
(333, 114.52433800697327)
(334, 115.55755686759949)
(335, 116.58574104309082)
(336, 117.60660290718079)
(337, 118.65232825279236)
(338, 119.70525622367859)
(339, 120.84426403045654)
(340, 121.9854199886322)
(341, 123.19009399414062)
(342, 124.31953191757202)
(343, 125.46595597267151)
(344, 126.60452604293823)
(345, 127.70255088806152)
(346, 128.80254006385803)
(347, 129.91553711891174)
(348, 131.03532886505127)
(349, 132.1655080318451)
(350, 133.30829620361328)
(351, 134.45734119415283)
(352, 135.60238814353943)
(353, 136.75472927093506)
(354, 137.9089629650116)
(355, 139.07820010185242)
(356, 140.25049996376038)
(357, 141.43911004066467)
(358, 142.62330102920532)
(359, 143.81030225753784)
(360, 145.0058810710907)
(361, 146.20431995391846)
(362, 147.416766166687)
(363, 148.63925504684448)
(364, 149.8822410106659)
(365, 151.1303451061

(651, 838.2390859127045)
(652, 842.0046870708466)
(653, 845.8709080219269)
(654, 849.8117971420288)
(655, 853.7270171642303)
(656, 857.6504411697388)
(657, 861.6074552536011)
(658, 865.5559830665588)
(659, 869.4952440261841)
(660, 873.4371280670166)
(661, 877.3882632255554)
(662, 881.2732589244843)
(663, 885.21235704422)
(664, 889.2057180404663)
(665, 893.2800459861755)
(666, 897.3132810592651)
(667, 901.3670210838318)
(668, 905.4210741519928)
(669, 909.4239242076874)
(670, 913.4427480697632)
(671, 917.4126839637756)
(672, 921.4608502388)
(673, 925.5350382328033)
(674, 929.7029302120209)
(675, 933.8266983032227)
(676, 937.9003839492798)
(677, 941.942939043045)
(678, 945.9832260608673)
(679, 950.0183210372925)
(680, 954.1041388511658)
(681, 958.617534160614)
(682, 962.9558329582214)
(683, 967.2821621894836)
(684, 971.5524060726166)
(685, 975.8213510513306)
(686, 980.0908799171448)
(687, 984.2645471096039)
(688, 988.4609911441803)
(689, 992.6194000244141)
(690, 996.8082809448242)
(691, 1

(972, 2802.7690970897675)
(973, 2811.849442958832)
(974, 2821.0792832374573)
(975, 2830.537930250168)
(976, 2839.921467065811)
(977, 2849.3079771995544)
(978, 2858.615399122238)
(979, 2867.882728099823)
(980, 2877.2468230724335)
(981, 2886.5895199775696)
(982, 2895.911281108856)
(983, 2905.3213810920715)
(984, 2914.7925181388855)
(985, 2924.1983709335327)
(986, 2933.5617802143097)
(987, 2943.145203113556)
(988, 2952.5555460453033)
(989, 2962.020059108734)
(990, 2971.502038002014)
(991, 2980.9810342788696)
(992, 2990.491399049759)
(993, 2999.9407291412354)
(994, 3009.5310492515564)
(995, 3018.981581926346)
(996, 3028.4512560367584)
(997, 3037.8518030643463)
(998, 3047.5652101039886)
(999, 3057.1097271442413)
(1000, 3066.646558046341)

od:
[0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 5, 2, 2, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 5, 2, 1, 1, 1, 3, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 2, 4, 1, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 1, 2, 4, 2,

In [2]:
# verify narrow version, p = 5
import time

def p5RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 290
    ans=0
    for a in [1..n+1]:
        term=rmnjntau(a)
        ans=ans+term*x^a
    return D(ans)

def recipP5(n):
    ans=L(1/p5RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p5RecipPowerConstantTerm(k):
    g=recipP5(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

A037453=[]
zeros=[]
od=[]
start=time.time()
for a in [1..5000]: #default precision must be > max a.
    c=p5RecipPowerConstantTerm(a)
    o5=order(5,c)
    if o5==0:zeros=zeros+[a]
    od=od+[o5]
    c3=census(3,a.digits(5))
    c4=census(4,a.digits(5))
    if c3==0:
        if c4==0:
              A037453=A037453+[a]
    finish=time.time()
    print((a,finish-start))
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(A037453);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)
# ps is A037453		Positive numbers whose base-5 representation contains no 3 or 4.

(1, 0.05638289451599121)
(2, 0.0745549201965332)
(3, 0.0919947624206543)
(4, 0.10907578468322754)
(5, 0.1233527660369873)
(6, 0.15314269065856934)
(7, 0.18699169158935547)
(8, 0.21436786651611328)
(9, 0.23920583724975586)
(10, 0.2577977180480957)
(11, 0.27852487564086914)
(12, 0.2960057258605957)
(13, 0.312349796295166)
(14, 0.33107471466064453)
(15, 0.35708069801330566)
(16, 0.38167500495910645)
(17, 0.39965081214904785)
(18, 0.4176177978515625)
(19, 0.4406278133392334)
(20, 0.4577138423919678)
(21, 0.4806637763977051)
(22, 0.5027267932891846)
(23, 0.5210797786712646)
(24, 0.5418517589569092)
(25, 0.5614500045776367)
(26, 0.5808267593383789)
(27, 0.6002869606018066)
(28, 0.6232597827911377)
(29, 0.6447968482971191)
(30, 0.6670567989349365)
(31, 0.6895859241485596)
(32, 0.7111999988555908)
(33, 0.7350509166717529)
(34, 0.7570219039916992)
(35, 0.7793569564819336)
(36, 0.8022799491882324)
(37, 0.8253068923950195)
(38, 0.8492007255554199)
(39, 0.8789770603179932)
(40, 0.9061298370361328)

(328, 111.53986883163452)
(329, 112.56806683540344)
(330, 113.60706567764282)
(331, 114.65333676338196)
(332, 115.72058176994324)
(333, 116.78554487228394)
(334, 117.83561491966248)
(335, 118.8648328781128)
(336, 120.15385389328003)
(337, 121.37147879600525)
(338, 122.53897762298584)
(339, 123.68262076377869)
(340, 124.87558889389038)
(341, 125.99531865119934)
(342, 127.11763381958008)
(343, 128.26407980918884)
(344, 129.38381171226501)
(345, 130.50938081741333)
(346, 131.6490068435669)
(347, 132.82000088691711)
(348, 133.99754285812378)
(349, 135.14895367622375)
(350, 136.37883377075195)
(351, 137.55263686180115)
(352, 138.74906873703003)
(353, 139.98391771316528)
(354, 141.1886489391327)
(355, 142.38469290733337)
(356, 143.59529876708984)
(357, 144.8186550140381)
(358, 146.06925797462463)
(359, 147.25120282173157)
(360, 148.50185704231262)
(361, 149.72644686698914)
(362, 150.99769186973572)
(363, 152.28850483894348)
(364, 153.55560398101807)
(365, 154.8161187171936)
(366, 156.0729987

(652, 867.0833127498627)
(653, 870.7274677753448)
(654, 874.4004187583923)
(655, 878.0824220180511)
(656, 881.7331738471985)
(657, 885.4450256824493)
(658, 889.1649677753448)
(659, 892.8603909015656)
(660, 896.583370923996)
(661, 900.3064796924591)
(662, 904.0577459335327)
(663, 907.8285918235779)
(664, 911.6307137012482)
(665, 915.4316086769104)
(666, 919.262021780014)
(667, 923.0561819076538)
(668, 926.8520538806915)
(669, 930.6683547496796)
(670, 934.4621489048004)
(671, 938.2714407444)
(672, 942.0864927768707)
(673, 946.7444260120392)
(674, 953.5998888015747)
(675, 959.0553798675537)
(676, 963.4623727798462)
(677, 967.6009459495544)
(678, 971.5898020267487)
(679, 975.6110436916351)
(680, 979.6419966220856)
(681, 983.6879518032074)
(682, 987.7053868770599)
(683, 991.5469980239868)
(684, 995.4733037948608)
(685, 999.3732407093048)
(686, 1003.1872277259827)
(687, 1007.1166956424713)
(688, 1010.9716060161591)
(689, 1014.826653957367)
(690, 1018.7239148616791)
(691, 1022.6838090419769)


(973, 2643.494791984558)
(974, 2650.663414001465)
(975, 2657.84477186203)
(976, 2665.0156569480896)
(977, 2672.231199026108)
(978, 2679.4504809379578)
(979, 2686.767033815384)
(980, 2694.036118030548)
(981, 2701.312604904175)
(982, 2708.5587599277496)
(983, 2715.8451216220856)
(984, 2723.1641516685486)
(985, 2730.4957609176636)
(986, 2737.879690885544)
(987, 2745.1780438423157)
(988, 2752.561636686325)
(989, 2759.984778881073)
(990, 2767.388361930847)
(991, 2774.8482189178467)
(992, 2782.2650458812714)
(993, 2789.7268109321594)
(994, 2797.179054737091)
(995, 2804.72580575943)
(996, 2812.2391867637634)
(997, 2819.7549197673798)
(998, 2827.2356708049774)
(999, 2834.779441833496)
(1000, 2842.328920841217)
(1001, 2849.9058108329773)
(1002, 2857.5661346912384)
(1003, 2865.25297164917)
(1004, 2872.8984858989716)
(1005, 2880.5254678726196)
(1006, 2888.16521191597)
(1007, 2895.82150888443)
(1008, 2903.48295879364)
(1009, 2911.1446118354797)
(1010, 2918.815702676773)
(1011, 2926.5953118801117)


(1287, 5702.492090702057)
(1288, 5715.047073841095)
(1289, 5727.558562755585)
(1290, 5740.120630025864)
(1291, 5752.664887666702)
(1292, 5765.224733829498)
(1293, 5777.850870609283)
(1294, 5790.426394701004)
(1295, 5803.186564683914)
(1296, 5815.783255815506)
(1297, 5828.492456912994)
(1298, 5841.209909915924)
(1299, 5853.965278863907)
(1300, 5866.761909008026)
(1301, 5879.549810886383)
(1302, 5892.326563835144)
(1303, 5905.122882843018)
(1304, 5917.969120979309)
(1305, 5930.865749835968)
(1306, 5943.808241844177)
(1307, 5956.738115787506)
(1308, 5969.546183824539)
(1309, 5982.450362920761)
(1310, 5995.447722911835)
(1311, 6008.372023820877)
(1312, 6021.315250873566)
(1313, 6034.34588599205)
(1314, 6047.307140827179)
(1315, 6060.358440876007)
(1316, 6073.416128873825)
(1317, 6086.481590747833)
(1318, 6099.486025810242)
(1319, 6112.61213684082)
(1320, 6125.827404022217)
(1321, 6138.937106847763)
(1322, 6152.161935806274)
(1323, 6165.498495817184)
(1324, 6178.775781869888)
(1325, 6192.09

(1602, 10739.966977834702)
(1603, 10759.39180970192)
(1604, 10778.765343904495)
(1605, 10798.71576499939)
(1606, 10818.251281738281)
(1607, 10837.834572076797)
(1608, 10857.365855932236)
(1609, 10876.960867881775)
(1610, 10896.618244886398)
(1611, 10916.394518613815)
(1612, 10936.084850788116)
(1613, 10955.793193817139)
(1614, 10975.4641289711)
(1615, 10995.365761041641)
(1616, 11014.981306791306)
(1617, 11034.810834884644)
(1618, 11054.637138843536)
(1619, 11074.83213686943)
(1620, 11094.980000972748)
(1621, 11114.883801698685)
(1622, 11134.754234790802)
(1623, 11154.721960783005)
(1624, 11174.643536806107)
(1625, 11194.818034887314)
(1626, 11214.78255367279)
(1627, 11234.779054880142)
(1628, 11254.834444761276)
(1629, 11274.966540813446)
(1630, 11295.246755838394)
(1631, 11315.508624792099)
(1632, 11335.571802854538)
(1633, 11355.878890037537)
(1634, 11376.321600914001)
(1635, 11396.822275876999)
(1636, 11417.226927757263)
(1637, 11437.586414813995)
(1638, 11457.881577968597)
(1639, 

KeyboardInterrupt: 

In [3]:
print();print("od:");print(od);print()
ps=positions(0,od)
vno=vector(A037453);vps=vector(ps)
print("vno-vps:");print();print(vno-vps)
# ps is A037453		Positive numbers whose base-5 representation contains no 3 or 4.


od:
[0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 5, 2, 2, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 5, 2, 1, 1, 1, 3, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 2, 4, 1, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 1, 2, 4, 2, 1, 1, 2, 2, 3, 3, 2, 3, 3, 3, 4, 2, 3, 3, 3, 1, 1, 2, 2, 2, 1, 1, 2, 3, 2, 1, 1, 2, 2, 4, 2, 3, 4, 3, 3, 2, 4, 3, 3, 3, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 2, 2, 2, 1, 1, 3, 2, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 3, 2, 1, 1, 1, 5, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 4, 1, 1, 1, 3, 2, 1, 1, 1, 5, 2, 1, 1, 1, 3, 3, 3, 2, 2, 5, 3, 3, 2, 2, 4, 3, 1, 1, 1, 3, 2, 1, 1, 1, 4, 2, 1, 1, 1, 3, 4, 2, 3, 2, 5, 3, 2, 3, 2, 4, 3, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 3, 2, 1, 1, 1, 3, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 2, 3, 1, 1, 1, 2, 3, 1, 1, 1